<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
using Pkg; Pkg.activate(".")

  Activating project at `C:\Users\lukas\Desktop\AMO\Team_assignment_AMO`


In [2]:
using JuMP,HiGHS,DataFrames

In [286]:
nP=1:3                                  #number of generation units
nW=1:1                                  #number of stochastic generation units
nt=1:1                                  #number of time intervalls 
nS=1:2                                  #number of scenarios
nL=1:2                                  #number of loads
nB=1:1                                  #number of buses
C=[100, 80, 50]                         #unit cost of generation
C_RU=[20, 10, 15]                       #cost of up reserve commitment
C_RD=[25, 15, 15]                       #cost of down reserve commitment
C_U=[10, 15, 20]                        #cost of up reserve commitment
C_D=[30, 10, 15]                        #cost of down reserve commitment
pi=[0.7, 0.3]                           #probabilities of the scenarios
V_LOL=[20, 25]                          #value of the lost load
C_cur=[15]                              #cost of curtailment at turbine
L= 90 .+ 30 * randn(2, 1)               #loads for every time intervall
W_realized = 70 .+ 15 * randn(1, 2, 1)  #create random scenarios for four time points
P_max=[70, 80, 150];                    #maximum production capacity

[85.4903355623685; 90.28708638969704;;][55.30193164463399 57.98800331952636;;;]

In [287]:
data= Dict(
    :nP => nP,
    :nW => nW,
    :nt => nt,
    :nS => nS,
    :nL => nL,
    :C => C,
    :C_RU => C_RU,
    :C_RD => C_RD,
    :P_max => P_max,
    :C_U => C_U,
    :C_D => C_D,
    :pi => pi,
    :V_LOL => V_LOL,
    :C_cur => C_cur,
    :L => L,
    :W_realized => W_realized,
    :P_max => P_max
    );
print(W_realized[1,2,1])

57.98800331952636

In [295]:
function init_variables(model::JuMP.Model)
    P=@variable(model, P[p in data[:nP], t in data[:nt]] >=0)
    R_U=@variable(model, R_U[p in data[:nP], t in data[:nt]] >=0)
    R_D=@variable(model, R_D[p in data[:nP], t in data[:nt]] >=0)
    r_U=@variable(model, r_U[p in data[:nP], s in data[:nS], t in data[:nt]] >=0)
    r_D=@variable(model, r_D[p in data[:nP], s in data[:nS], t in data[:nt]] >=0)
    L_Shed=@variable(model, L_Shed[l in data[:nL], s in data[:nS], t in data[:nt]] >=0)
    W_spill=@variable(model, W_spill[w in data[:nW], s in data[:nS], t in data[:nt]] >=0)
    W_s=@variable(model, W_s[w in data[:nW], t in data[:nt]] >=0)
    
    vars= Dict(
        :P=> P,
        :R_U => R_U,
        :R_D => R_D,
        :r_U => r_U,
        :r_D => r_D,
        :L_Shed => L_Shed,
        :W_spill => W_spill,
        :W_s => W_s
    )
    
    return vars
end;

In [299]:
function init_constraints(model::JuMP.Model, data::Dict, vars::Dict)
    @constraints(model, begin 
            c1[p in data[:nP], t in data[:nt]], vars[:P][p, t]+ vars[:R_U][p, t]<= P_max[p]
            c2[p in data[:nP] ,t in data[:nt]], vars[:P][p, t]- vars[:R_D][p, t]>= 0
            c3[p in data[:nP], t in data[:nt], s in data[:nS]], vars[:r_U][p, s, t] <= vars[:R_U][p,t]
            c4[p in data[:nP], t in data[:nt], s in data[:nS]], vars[:r_D][p, s, t] <= vars[:R_D][p,t]  
            Power_balance_day_ahead[t in data[:nt]], 
            sum(vars[:P][p,t] for p in data[:nP]) + sum(vars[:W_s][w, t] for w in data[:nW]) - sum(L[l, t] for l in data[:nL]) == 0
            Power_balance_at_stage[s in data[:nS], t in data[:nt]], 
            sum(vars[:r_U][p, s, t] for p in data[:nP]) - sum(vars[:r_D][p, s, t] for p in data[:nP]) - 
            sum(vars[:L_Shed][l, s, t] for l in data[:nL]) + sum(data[:W_realized][w,s,t] -vars[:W_s][w, t] for w in data[:nW])== 0      
    end)
end;

In [300]:
function init_obj_function(model, data, vars)
    @objective(model, Min, sum(vars[:P][p,t]* C[p] + vars[:R_U][p,t] * C_RU[p] + vars[:R_D][p,t] * C_RD[p] for p in data[:nP], t in data[:nt]) +
    sum(pi[s] * (sum(data[:C_U][p] * vars[:r_U][p, s ,t] + data[:C_D][p] * vars[:r_D][p, s, t] for p in data[:nP], t in data[:nt]) +
                sum(data[:C_cur][w] * vars[:W_spill][w, s, t] for w in data[:nW], t in data[:nt]) 
                + sum(data[:V_LOL][l] * vars[:L_Shed][l, s, t] for l in data[:nL], t in data[:nt])) for s in data[:nS])
    )
end;

In [301]:
model=Model(HiGHS.Optimizer)
vars=init_variables(model)
init_constraints(model, data, vars)
init_obj_function(model, data, vars)
print(model)

In [302]:
optimize!(model)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
21 rows, 24 cols, 56 nonzeros
17 rows, 20 cols, 48 nonzeros
17 rows, 20 cols, 48 nonzeros
Presolve : Reductions: rows 17(-4); columns 20(-8); elements 48(-10)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -1.0999192113e+02 Ph1: 5(6); Du: 2(109.992) 0s
         10     3.2021773867e+03 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 10
Objective value     :  3.2021773867e+03
HiGHS run time      :          0.00


In [308]:
# Print the values of the variables
println("Variable values:")
println("P:")
for p in data[:nP], t in data[:nt]
    println("P[$p]: ", value(vars[:P][p, t]))
end

println("\nR_U:")
for p in data[:nP], t in data[:nt]
    println("R_U[$p]: ", value(vars[:R_U][p, t]))
end

println("\nR_D:")
for p in data[:nP], t in data[:nt]
    println("R_D[$p]: ", value(vars[:R_D][p, t]))
end

println("\nL_Shed:")
for l in data[:nL], s in data[:nS], t in data[:nt]
    println("L_Shed[$l, $s, $t]: ", value(vars[:L_Shed][l, s, t]))
end

println("\nW_spill:")
for w in data[:nW], s in data[:nS], t in data[:nt]
    println("W_spill[$w, $s, $t]: ", value(vars[:W_spill][w, s, t]))
end

println("\nW_realized:")
for w in data[:nW], s in data[:nS], t in data[:nt]
    println("W_realized[$w, $s, $t]: ", value(data[:W_realized][w, s, t]))
end

println("\nW_s:")
for w in data[:nW], t in data[:nt]
    println("W_s[$w, $t]: ", value(vars[:W_s][w, t]))
end

println("\nr_U:")
for p in data[:nP], s in data[:nS], t in data[:nt]
    println("r_U[$p, $s, $t]: ", value(vars[:r_U][p, s, t]))
end

println("\nr_D:")
for p in data[:nP], s in data[:nS], t in data[:nt]
    println("r_D[$p, $s, $t]: ", value(vars[:r_D][p, s, t]))
end

println("\nLoad")
for l in data[:nL]
    print("L[$l]:", data[:L][l])
end

print("\n\nsum:", sum(data[:L]))



Variable values:
P:
P[1]: 0.0
P[2]: -0.0
P[3]: 0.0

R_U:
R_U[1]: 40.475490307431556
R_U[2]: 80.0
R_U[3]: 0.0

R_D:
R_D[1]: 0.0
R_D[2]: 0.0
R_D[3]: 0.0

L_Shed:
L_Shed[1, 1, 1]: 0.0
L_Shed[1, 2, 1]: 0.0
L_Shed[2, 1, 1]: 0.0
L_Shed[2, 2, 1]: 0.0

W_spill:
W_spill[1, 1, 1]: 0.0
W_spill[1, 2, 1]: 0.0

W_realized:
W_realized[1, 1, 1]: 55.30193164463399
W_realized[1, 2, 1]: 57.98800331952636

W_s:
W_s[1, 1]: 175.77742195206554

r_U:
r_U[1, 1, 1]: 40.475490307431556
r_U[1, 2, 1]: 40.475490307431556
r_U[2, 1, 1]: 80.0
r_U[2, 2, 1]: 77.31392832510763
r_U[3, 1, 1]: -0.0
r_U[3, 2, 1]: 0.0

r_D:
r_D[1, 1, 1]: 0.0
r_D[1, 2, 1]: 0.0
r_D[2, 1, 1]: 0.0
r_D[2, 2, 1]: 0.0
r_D[3, 1, 1]: 0.0
r_D[3, 2, 1]: 0.0

Load
L[1]:85.4903355623685L[2]:90.28708638969704

sum:175.77742195206554